<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    55
1     9
Name: InPortfolio, dtype: int64

In [6]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].ewm(span = mid_window, adjust = False).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    stock_df['Trend'] = 'Down'
    stock_df.loc[stock_df['MACD_12_26_9'] > stock_df['MACDs_12_26_9'], 'Trend'] = 'Up'
    stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    stock_df['EOM'] = '-ve'
    stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    #print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [8]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM
Date,,,,,,,,,,,,
2022-12-21 00:00:00+05:30,18199.0,^NSEI,4.13,0.59,0.18,18812.0,15294.0,82.58,17.42,Down,43.0,-ve
2022-12-22 00:00:00+05:30,18127.0,^NSEI,3.68,0.55,0.11,18812.0,15294.0,80.54,19.46,Down,40.0,-ve
2022-12-23 00:00:00+05:30,17807.0,^NSEI,1.82,0.51,0.01,18812.0,15294.0,71.43,28.57,Down,32.0,-ve
2022-12-26 00:00:00+05:30,18015.0,^NSEI,2.98,0.47,-0.08,18812.0,15294.0,77.33,22.67,Down,41.0,-ve
2022-12-27 00:00:00+05:30,18132.0,^NSEI,3.62,0.45,-0.14,18812.0,15294.0,80.68,19.32,Down,45.0,-ve


In [9]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [10]:
df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)

,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
1,2022-12-27 00:00:00+05:30,450.0,ICICIPRULI.NS,-12.37,-1.36,-1.26,615.0,440.0,5.94,94.06,Up,42.0,-ve,640.0,42.0
2,2022-12-27 00:00:00+05:30,696.0,MOTILALOFS.NS,-8.23,-0.92,0.00,960.0,654.0,13.89,86.11,Up,55.0,+ve,930.0,34.0
6,2022-12-27 00:00:00+05:30,520.0,JUBLFOOD.NS,-10.41,-1.03,-1.27,785.0,463.0,17.59,82.41,Up,40.0,-ve,643.0,24.0
56,2022-12-27 00:00:00+05:30,3477.0,DIVISLAB.NS,-6.53,-0.86,0.29,4643.0,3266.0,15.33,84.67,Up,57.0,+ve,3560.0,2.0
57,2022-12-27 00:00:00+05:30,1096.0,CIPLA.NS,4.97,0.57,0.09,1173.0,863.0,75.11,24.89,Up,44.0,-ve,1092.0,-0.0


In [11]:
# portfolio condition
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
1,2022-12-27 00:00:00+05:30,450.0,ICICIPRULI.NS,-12.37,-1.36,-1.26,615.0,440.0,5.94,94.06,Up,42.0,-ve,640.0,42.0
2,2022-12-27 00:00:00+05:30,696.0,MOTILALOFS.NS,-8.23,-0.92,0.00,960.0,654.0,13.89,86.11,Up,55.0,+ve,930.0,34.0
3,2022-12-27 00:00:00+05:30,1241.0,SBILIFE.NS,2.14,0.25,-0.32,1332.0,1023.0,70.57,29.43,Down,46.0,-ve,1607.0,29.0
13,2022-12-27 00:00:00+05:30,570.0,HDFCLIFE.NS,0.85,0.18,0.35,672.0,502.0,39.88,60.12,Down,49.0,-ve,681.0,19.0
14,2022-12-27 00:00:00+05:30,3934.0,NAUKRI.NS,-6.02,-0.26,-0.20,5667.0,3363.0,24.79,75.21,Down,44.0,-ve,4677.0,19.0
18,2022-12-27 00:00:00+05:30,1066.0,MUTHOOTFIN.NS,-5.92,-0.53,-0.19,1517.0,956.0,19.59,80.41,Down,44.0,-ve,1217.0,14.0
25,2022-12-27 00:00:00+05:30,1515.0,INFY.NS,-2.83,-0.26,-0.45,1898.0,1351.0,29.95,70.05,Down,42.0,-ve,1715.0,13.0
26,2022-12-27 00:00:00+05:30,1010.0,TECHM.NS,-11.52,-1.14,-0.67,1750.0,935.0,9.15,90.85,Down,41.0,-ve,1139.0,13.0
56,2022-12-27 00:00:00+05:30,3477.0,DIVISLAB.NS,-6.53,-0.86,0.29,4643.0,3266.0,15.33,84.67,Up,57.0,+ve,3560.0,2.0


In [12]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 3


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
6,2022-12-27 00:00:00+05:30,520.0,JUBLFOOD.NS,-10.41,-1.03,-1.27,785.0,463.0,17.59,82.41,Up,40.0,-ve,643.0,24.0
10,2022-12-27 00:00:00+05:30,1110.0,HAVELLS.NS,-10.78,-0.80,-1.50,1400.0,1068.0,12.61,87.39,Down,29.0,-ve,1328.0,20.0
16,2022-12-27 00:00:00+05:30,2504.0,TITAN.NS,0.71,0.12,-0.78,2770.0,1929.0,68.32,31.68,Down,40.0,-ve,2967.0,18.0


In [13]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     22.702703
2     18.378378
3     15.675676
13    10.270270
14    10.270270
18     7.567568
25     7.027027
26     7.027027
56     1.081081
Name: Upside%, dtype: float64

In [14]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
0,2022-12-27 00:00:00+05:30,437.0,GODREJIND.NS,-7.44,-0.64,-0.45,644.0,405.0,13.35,86.65,Down,44.0,-ve,638.0,46.0
4,2022-12-27 00:00:00+05:30,239.0,ZEEL.NS,-8.06,-0.38,-1.56,325.0,206.0,27.77,72.23,Down,32.0,-ve,303.0,27.0
5,2022-12-27 00:00:00+05:30,394.0,TATAMOTORS.NS,-8.76,-0.69,-1.44,525.0,372.0,14.48,85.52,Down,38.0,-ve,495.0,26.0
6,2022-12-27 00:00:00+05:30,520.0,JUBLFOOD.NS,-10.41,-1.03,-1.27,785.0,463.0,17.59,82.41,Up,40.0,-ve,643.0,24.0
7,2022-12-27 00:00:00+05:30,6505.0,BAJFINANCE.NS,-4.40,-0.38,-1.02,7824.0,5265.0,48.44,51.56,Down,41.0,-ve,8015.0,23.0
8,2022-12-27 00:00:00+05:30,901.0,ICICIBANK.NS,7.22,0.83,-0.33,953.0,650.0,82.72,17.28,Down,45.0,-ve,1094.0,21.0
9,2022-12-27 00:00:00+05:30,3202.0,EICHERMOT.NS,1.64,0.38,-1.30,3850.0,2241.0,59.71,40.29,Down,40.0,-ve,3884.0,21.0


In [15]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 1


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
34,2022-12-27 00:00:00+05:30,387.0,WIPRO.NS,-14.28,-1.33,-0.76,714.0,375.0,3.42,96.58,Down,42.0,-ve,425.0,10.0


In [16]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Premium%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Dev%_200', ascending=True)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
35,2022-12-27 00:00:00+05:30,2669.0,HDFC.NS,8.90,0.94,0.46,2729.0,2053.0,91.14,8.86,Down,53.0,+ve,2940.0,10.0
31,2022-12-27 00:00:00+05:30,1631.0,HDFCBANK.NS,9.76,1.01,0.57,1662.0,1281.0,91.89,8.11,Down,56.0,+ve,1813.0,11.0
61,2022-12-27 00:00:00+05:30,2124.0,LT.NS,12.21,1.43,0.53,2195.0,1453.0,90.44,9.56,Down,54.0,-ve,2036.0,-4.0
21,2022-12-27 00:00:00+05:30,133.0,FEDERALBNK.NS,16.62,1.75,0.00,138.0,80.0,91.29,8.71,Down,50.0,-ve,152.0,14.0
20,2022-12-27 00:00:00+05:30,932.0,AXISBANK.NS,18.00,1.90,1.01,950.0,625.0,94.52,5.48,Down,58.0,+ve,1060.0,14.0
45,2022-12-27 00:00:00+05:30,111.0,TATASTEEL.NS,23.50,2.22,0.00,116.0,44.0,93.26,6.74,Down,55.0,-ve,118.0,6.0
59,2022-12-27 00:00:00+05:30,289.0,INDIANB.NS,39.40,4.35,1.51,300.0,127.0,93.38,6.62,Down,55.0,-ve,284.0,-2.0


In [17]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
52,2022-12-27 00:00:00+05:30,3260.0,TCS.NS,-1.67,-0.18,-0.15,3965.0,2974.0,28.81,71.19,Down,46.0,-ve,3385.0,4.0
39,2022-12-27 00:00:00+05:30,3113.0,ASIANPAINT.NS,-1.44,-0.22,-0.48,3566.0,2576.0,54.20,45.80,Down,49.0,-ve,3361.0,8.0
49,2022-12-27 00:00:00+05:30,4075.0,DMART.NS,-1.79,-0.27,-0.59,4731.0,3231.0,56.24,43.76,Down,52.0,-ve,4282.0,5.0
27,2022-12-27 00:00:00+05:30,1821.0,KOTAKBANK.NS,-1.31,-0.05,-0.69,1966.0,1641.0,55.35,44.65,Down,35.0,-ve,2055.0,13.0
7,2022-12-27 00:00:00+05:30,6505.0,BAJFINANCE.NS,-4.40,-0.38,-1.02,7824.0,5265.0,48.44,51.56,Down,41.0,-ve,8015.0,23.0
24,2022-12-27 00:00:00+05:30,8305.0,MARUTI.NS,-1.97,-0.06,-1.16,9528.0,6725.0,56.38,43.62,Down,35.0,-ve,9356.0,13.0
32,2022-12-27 00:00:00+05:30,1538.0,BAJAJFINSV.NS,-2.43,-0.06,-1.17,1836.0,1093.0,59.85,40.15,Down,37.0,-ve,1691.0,10.0
30,2022-12-27 00:00:00+05:30,208.0,TATAPOWER.NS,-7.69,-0.44,-1.36,287.0,195.0,13.80,86.20,Down,39.0,-ve,232.0,12.0
4,2022-12-27 00:00:00+05:30,239.0,ZEEL.NS,-8.06,-0.38,-1.56,325.0,206.0,27.77,72.23,Down,32.0,-ve,303.0,27.0
23,2022-12-27 00:00:00+05:30,1985.0,DEEPAKNTR.NS,-6.27,-0.28,-1.64,2650.0,1713.0,29.06,70.94,Down,37.0,-ve,2260.0,14.0


In [18]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 0


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%


In [19]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
43,2022-12-27 00:00:00+05:30,2028.0,INDIGO.NS,7.8,0.64,0.78,2265.0,1549.0,66.86,33.14,Down,58.0,+ve,2176.0,7.0


In [20]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 5


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
12,2022-12-27 00:00:00+05:30,80.0,IBREALEST.NS,-9.83,-1.12,0.00,174.0,60.0,17.76,82.24,Down,46.0,-ve,95.0,19.0
52,2022-12-27 00:00:00+05:30,3260.0,TCS.NS,-1.67,-0.18,-0.15,3965.0,2974.0,28.81,71.19,Down,46.0,-ve,3385.0,4.0
0,2022-12-27 00:00:00+05:30,437.0,GODREJIND.NS,-7.44,-0.64,-0.45,644.0,405.0,13.35,86.65,Down,44.0,-ve,638.0,46.0
34,2022-12-27 00:00:00+05:30,387.0,WIPRO.NS,-14.28,-1.33,-0.76,714.0,375.0,3.42,96.58,Down,42.0,-ve,425.0,10.0
47,2022-12-27 00:00:00+05:30,4390.0,LTIM.NS,-11.09,-1.22,-1.19,7506.0,3796.0,16.02,83.98,Down,42.0,-ve,4654.0,6.0
